In [71]:
%matplotlib inline

# base
import sys
import os
import math
import random
import logging
import IPython
from pathlib import Path
from datetime import datetime
from timeit import default_timer as timer

# image processing
import cv2
from PIL import Image

#ml
import numpy as np
import scipy as sp
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.models import resnet34

In [72]:
print('Python version: {}'. format(sys.version))
print('SciPy version: {}'. format(sp.__version__)) 
print('NumPy version: {}'. format(np.__version__))
print('scikit-learn version: {}'. format(sklearn.__version__))
print('pandas version: {}'. format(pd.__version__))
print('matplotlib version: {}'. format(matplotlib.__version__))
print('IPython version: {}'. format(IPython.__version__)) 
print('OpenCV version: {}'. format(cv2.__version__)) 
print('Torch version: {}'. format(torch.__version__))
print('Available GPUs: {}'.format(torch.cuda.device_count()))
if torch.cuda.is_available:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print("Torch device: {}".format(device))

Python version: 3.8.1 (default, Jan  8 2020, 22:29:32) 
[GCC 7.3.0]
SciPy version: 1.4.1
NumPy version: 1.18.1
scikit-learn version: 0.22.2.post1
pandas version: 1.0.3
matplotlib version: 3.2.1
IPython version: 7.13.0
OpenCV version: 4.2.0
Torch version: 1.4.0
Available GPUs: 1
Torch device: cuda


In [73]:
 SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [74]:
def null_collate(batch):
    batch_size = len(batch)
    images = np.array([x[0] for x in batch])
    images = torch.from_numpy(images)
    
    labels = np.array([x[1] for x in batch])
    labels = torch.from_numpy(labels)
    labels = labels.unsqueeze(1)

    assert(images.shape[0] == labels.shape[0] == batch_size)
    
    return images, labels

In [75]:
CHESTXRAYS_PATH = Path('./input/chestxrays')
class ChestXRaysDataset(Dataset):
    def __init__(self, size=128, augment=None):
        super(ChestXRaysDataset, self).__init__()
        print('ChestXRaysDataset initialized with size={}, augment={}'.format(size, augment))
        print('Dataset is located in {}'.format(CHESTXRAYS_PATH))
        self.size = size
        self.augment = augment
        
        train_dir = CHESTXRAYS_PATH / 'train'
        val_dir = CHESTXRAYS_PATH / 'val'
        test_dir = CHESTXRAYS_PATH / 'test'
        
        normal_cases = []
        pneumonia_cases = []
        for folder in [train_dir, val_dir, test_dir]:
            normal_cases.extend((folder / 'NORMAL').glob('*.jpeg'))
            pneumonia_cases.extend((folder / 'PNEUMONIA').glob('*.jpeg'))
            
        self.labels = np.concatenate((
            np.zeros(len(normal_cases)),
            np.ones(len(pneumonia_cases))
        )).reshape(-1, 1)
        self.images = np.concatenate((normal_cases, pneumonia_cases)).reshape(-1, 1)
        
        self.df = pd.DataFrame(np.concatenate((self.images, self.labels), axis=1), columns=['image', 'label'])
        
        del self.images

        print("Dataset: {}".format(self.df))
            

    @staticmethod
    def _load_image(path, size):
        img = Image.open(path)
        img = cv2.resize(np.array(img), (size, size), interpolation=cv2.INTER_AREA)
        if len(img.shape) == 2:
            img = np.expand_dims(img, axis=2)
            img = np.dstack([img, img, img])
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # size, size, chan -> chan, size, size
        img = np.transpose(img, axes=[2, 0, 1])
        
        return img
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = self._load_image(row['image'], self.size)
        label = row['label']        

        if self.augment is not None:
            img = self.augment(img)

        return img, label

    def __len__(self):
        return self.df.shape[0]

In [76]:
COVID19_PATH = Path('./input/covid-chestxray-dataset')
class COVID19Dataset(Dataset):
    def __init__(self, size=128, augment=None):
        super(ChestXRaysDataset, self).__init__()
        print('COVID19Dataset initialized with size={}, augment={}'.format(size, augment))
        print('Dataset is located in {}'.format(COVID19_PATH))
        self.size = size
        self.augment = augment
        
        train_dir = CHESTXRAYS_PATH / 'train'
        val_dir = CHESTXRAYS_PATH / 'val'
        test_dir = CHESTXRAYS_PATH / 'test'
        
        normal_cases = []
        pneumonia_cases = []
        for folder in [train_dir, val_dir, test_dir]:
            normal_cases.extend((folder / 'NORMAL').glob('*.jpeg'))
            pneumonia_cases.extend((folder / 'PNEUMONIA').glob('*.jpeg'))
            
        self.labels = np.concatenate((
            np.zeros(len(normal_cases)),
            np.ones(len(pneumonia_cases))
        )).reshape(-1, 1)
        self.images = np.concatenate((normal_cases, pneumonia_cases)).reshape(-1, 1)
        
        self.df = pd.DataFrame(np.concatenate((self.images, self.labels), axis=1), columns=['image', 'label'])
        
        del self.images

        print("Dataset: {}".format(self.df))
            

    @staticmethod
    def _load_image(path, size):
        img = Image.open(path)
        img = cv2.resize(np.array(img), (size, size), interpolation=cv2.INTER_AREA)
        if len(img.shape) == 2:
            img = np.expand_dims(img, axis=2)
            img = np.dstack([img, img, img])
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # size, size, chan -> chan, size, size
        img = np.transpose(img, axes=[2, 0, 1])
        
        return img
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = self._load_image(row['image'], self.size)
        label = row['label']        

        if self.augment is not None:
            img = self.augment(img)

        return img, label

    def __len__(self):
        return self.df.shape[0]

In [77]:
class Classifier(nn.Module):
    def __init__(self, num_classes=1, dropout=0.5):
        super(Classifier, self).__init__()
        resnet = resnet34(pretrained=True)
        
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool
        bottleneck_features = resnet.fc.in_features
        self.fc = nn.Sequential(
            nn.BatchNorm1d(bottleneck_features),
            nn.Dropout(dropout),
            nn.Linear(bottleneck_features, num_classes),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        # mean = MEAN
        # std = STD2
        x = x / 255.
        # x = torch.cat([
        #     (x[:, [0]] - mean[0]) / std[0],
        #     (x[:, [1]] - mean[1]) / std[1],
        #     (x[:, [2]] - mean[2]) / std[2],
        #     (x[:, [3]] - mean[3]) / std[3],
        # ], 1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x) 
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [78]:
class Accuracy(nn.Module):
    def __init__(self, threshold=0.5):
        super().__init__()
        self.threshold = threshold

    def forward(self, y_true, y_pred):
        preds = (y_pred > self.threshold).int()
        return (preds == y_true).sum().float() / len(preds)

In [79]:
class Trainer:
    def __init__(self, classifier, dataset, batch_size):
        self.classifier = classifier
        self.batch_size = batch_size
        self.size = size
        print('Trainer started with batch size: {} size: {}'.format(batch_size, size))

        self.optimizer = None
        self.scheduler = None

        self.train_dataset = dataset
        self.validation_dataset = dataset
        
       
        self.train_idx, self.validation_idx = train_test_split(
            list(range(len(self.train_dataset))),
            test_size=0.2,
            stratify=self.train_dataset.labels
        )

        loader_params = dict(
            batch_size=batch_size,
            num_workers=1,
            pin_memory=True,
            collate_fn=null_collate
        )
        self.train_loader = DataLoader(
            dataset=self.train_dataset,
            sampler=SubsetRandomSampler(self.train_idx),
            **loader_params
        )
        self.validation_loader = DataLoader(
            dataset=self.validation_dataset,
            sampler=SubsetRandomSampler(self.validation_idx),
            **loader_params
        )
        print('Train set: {}'.format(len(self.train_idx)))
        print('Validation set: {}'.format(len(self.validation_idx)))

        self.it_per_epoch = math.ceil(len(self.train_idx) / self.batch_size)
        print('Training with {} mini-batches per epoch'.format(self.it_per_epoch))

        
    def run(self, max_epochs=10):
        self.classifier = self.classifier.cuda()
        model = self.classifier

        lr = 0.2
        it = 0
        epoch = 0
        it_save = self.it_per_epoch * 5
        it_log = math.ceil(self.it_per_epoch / 5)
        it_smooth = self.it_per_epoch
        print("Logging performance every {} iter, smoothing every: {} iter".format(it_log, it_smooth))
        
        self.optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9, weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, 2 * self.it_per_epoch, gamma=0.5)

        criterion = nn.BCELoss()
        criterion = criterion.cuda()
        metrics = [Accuracy(), roc_auc_score]

        print("{}'".format(self.optimizer))
        print("{}'".format(self.scheduler))
        print("{}'".format(criterion))
        print("{}'".format(metrics))

        train_loss = 0
        train_roc = 0
        train_acc = 0

        print('                    |         VALID        |        TRAIN         |         ')
        print(' lr     iter  epoch | loss    roc    acc   | loss    roc    acc   |  time   ')
        print('------------------------------------------------------------------------------')

        start = timer()
        while epoch < max_epochs:
            smoothed_train_loss = 0
            smoothed_train_roc = 0
            smoothed_train_acc = 0
            smoothed_sum = 0

            for inputs, labels in self.train_loader:
                epoch = (it + 1) / self.it_per_epoch
                
                lr = self.scheduler.get_last_lr()[0]

                # checkpoint
                if it % it_save == 0 and it != 0:
                    self.save(model, self.optimizer, it, epoch)

                # training

                model.train()
                inputs = inputs.cuda().float()
                labels = labels.cuda().float()

                preds = model(inputs)
                loss = criterion(preds, labels)

                self.optimizer.zero_grad()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                self.optimizer.step()
                self.scheduler.step()
                
                with torch.no_grad():
                    batch_acc, batch_roc = [i(labels.cpu(), preds.cpu()).item() for i in metrics]

                batch_loss = loss.item()
                smoothed_train_loss += batch_loss
                smoothed_train_roc += batch_roc
                smoothed_train_acc += batch_acc
                smoothed_sum += 1
                if it % it_smooth == 0:
                    train_loss = smoothed_train_loss / smoothed_sum
                    train_roc = smoothed_train_roc / smoothed_sum
                    train_acc = smoothed_train_acc / smoothed_sum
                    smoothed_train_loss = 0
                    smoothed_train_roc = 0
                    smoothed_train_acc = 0
                    smoothed_sum = 0

                if it % it_log == 0:
                    print(
                        "{:5f} {:4d} {:5.1f} |                      | {:0.3f}  {:0.3f}  {:0.3f}  | {:6.2f}".format(
                            lr, it, epoch, batch_loss, batch_roc, batch_acc, timer() - start
                        ))

                it += 1

            # validation
            valid_loss, valid_m = self.do_valid(model, criterion, metrics)
            valid_acc, valid_roc = valid_m

            print(
                "{:5f} {:4d} {:5.1f} | {:0.3f}* {:0.3f}  {:0.3f}  | {:0.3f}* {:0.3f}  {:0.3f}  | {:6.2f}".format(
                    lr, it, epoch, valid_loss, valid_roc, valid_acc, train_loss, train_roc, train_acc, timer() - start
                ))

            # Data loader end
        # Training end

        self.save(model, self.optimizer, it, epoch)

    def do_valid(self, model, criterion, metrics):
        model.eval()
        valid_num = 0
        losses = []

        for inputs, labels in self.validation_loader:
            inputs = inputs.cuda().float()
            labels = labels.cuda().float()

            with torch.no_grad():
                preds = model(inputs)
                loss = criterion(preds, labels)
                m = [i(labels.cpu(), preds.cpu()).item() for i in metrics]

            valid_num += len(inputs)
            losses.append(loss.data.cpu().numpy())

        assert (valid_num == len(self.validation_loader.sampler))
        loss = np.array(losses).mean()
        return loss, m
    
    def save(self, model, optimizer, iter, epoch):
        torch.save(model.state_dict(), "{}_model.pth".format(iter))
        torch.save({
            "optimizer": optimizer.state_dict(),
            "iter": iter,
            "epoch": epoch
        }, "{}_optimizer.pth".format(iter))

In [80]:
batch_size = 64
size = 256
classifier = Classifier()

In [80]:
dataset = ChestXRaysDataset(size)
trainer = Trainer(classifier, dataset, batch_size)
trainer.run(max_epochs=10)

ChestXRaysDataset initialized with size=256, augment=None
Dataset is located in input/chestxrays
Dataset:                                                   image label
0       input/chestxrays/train/NORMAL/IM-0728-0001.jpeg     0
1     input/chestxrays/train/NORMAL/NORMAL2-IM-0698-...     0
2     input/chestxrays/train/NORMAL/NORMAL2-IM-0585-...     0
3     input/chestxrays/train/NORMAL/NORMAL2-IM-1302-...     0
4     input/chestxrays/train/NORMAL/IM-0656-0001-000...     0
...                                                 ...   ...
5851  input/chestxrays/test/PNEUMONIA/person1680_vir...     1
5852  input/chestxrays/test/PNEUMONIA/person1_virus_...     1
5853  input/chestxrays/test/PNEUMONIA/person83_bacte...     1
5854  input/chestxrays/test/PNEUMONIA/person43_virus...     1
5855  input/chestxrays/test/PNEUMONIA/person133_bact...     1

[5856 rows x 2 columns]
Trainer started with batch size: 64 size: 256
Train set: 4684
Validation set: 1172
Training with 74 mini-batches per epoch
Lo

## Citations
- http://www.cell.com/cell/fulltext/S0092-8674(18)30154-5